In [1]:
!pip install pandasql
import pandas as pd
import pandasql as psql
import numpy as np

In [3]:
deaths = pd.read_csv('/Users/benkelly/Downloads/Projects/COVID Overview/Data/28Jul2021Deaths.csv',header=0)
vax = pd.read_csv('/Users/benkelly/Downloads/Projects/COVID Overview/Data/28Jul2021Vax.csv',header=0)

In [4]:
#DeathPercent show's likelihood of dying if you contract COVID in each country 
#Overall at a certain snap shot, does not highlight changes overtime
query="""
SELECT location, date,total_cases,total_deaths, (total_deaths/total_cases)*100 as DeathPercent 
FROM deaths 
WHERE continent IS NOT NULL 
ORDER BY 1,2"""
deathPercent =  psql.sqldf(query)
deathPercent


,location,date,total_cases,total_deaths,DeathPercent
0,Afghanistan,2020-02-24,1.0,NaN,NaN
1,Afghanistan,2020-02-25,1.0,NaN,NaN
2,Afghanistan,2020-02-26,1.0,NaN,NaN
3,Afghanistan,2020-02-27,1.0,NaN,NaN
4,Afghanistan,2020-02-28,1.0,NaN,NaN
...,...,...,...,...,...
100599,Zimbabwe,2021-07-23,95686.0,2961.0,3.094497
100600,Zimbabwe,2021-07-24,97277.0,3050.0,3.135376
100601,Zimbabwe,2021-07-25,97894.0,3094.0,3.160561
100602,Zimbabwe,2021-07-26,99944.0,3173.0,3.174778


In [8]:
#Countries with proportionally highest infection rates
query="""
SELECT location,population,max(total_cases) AS highestCount, max((total_cases/population))*100 AS proportionalPercent 
FROM deaths 
WHERE continent IS NOT NULL 
GROUP BY location, population 
ORDER BY proportionalPercent DESC
"""
infPercent =  psql.sqldf(query)
infPercent.to_excel('/Users/benkelly/Downloads/Projects/COVID Overview/Data/3 Infection Percent.xlsx')

In [7]:
#Countries with proportionally highest infection rates by Date
query="""
SELECT location,population,date,max(total_cases) AS highestCount, 
       max((total_cases/population))*100 AS proportionalPercent 
FROM deaths 
WHERE continent IS NOT NULL 
GROUP BY location, population, date 
ORDER BY proportionalPercent DESC
"""
infPercentDate =  psql.sqldf(query)
infPercentDate.to_excel('/Users/benkelly/Downloads/Projects/COVID Overview/Data/4 Infection Percent Dates.xlsx')

In [6]:
#Proportional death rates
query="""
SELECT location, max(total_deaths) AS deathCount, max((total_deaths/population))*100 AS propDeathPercent 
FROM deaths 
WHERE continent IS NOT NULL 
GROUP BY location, population 
ORDER BY propDeathPercent DESC
"""
propDeath =  psql.sqldf(query)
propDeath

,location,deathCount,propDeathPercent
0,Peru,195890.0,0.594113
1,Hungary,30021.0,0.310765
2,Bosnia and Herzegovina,9686.0,0.295232
3,Czechia,30362.0,0.283519
4,San Marino,90.0,0.265189
...,...,...,...
217,Turkmenistan,NaN,NaN
218,Turks and Caicos Islands,NaN,NaN
219,Tuvalu,NaN,NaN
220,Vatican,NaN,NaN


In [32]:
#Proportional deaths by continent
query="""
SELECT location, max(total_deaths) AS deathCount, max((total_deaths/population))*100 AS propDeathPercent 
FROM deaths 
WHERE continent IS NULL 
GROUP BY location 
ORDER BY propDeathPercent DESC
"""
propDeathContinent =  psql.sqldf(query)
propDeathContinent

,location,deathCount,propDeathPercent
0,South America,1066539.0,0.247595
1,European Union,742507.0,0.166886
2,North America,913988.0,0.154371
3,Europe,1124496.0,0.150197
4,World,4118391.0,0.052835
5,Asia,852424.0,0.018372
6,Africa,159671.0,0.011910
7,Oceania,1258.0,0.002948
8,International,15.0,NaN


In [27]:
#Global Numbers
query="""
SELECT SUM(new_cases) AS total_cases ,SUM(new_deaths) AS total_deaths, 
       (SUM(new_deaths)/SUM(new_cases))*100 as DeathPercentage 
FROM deaths 
WHERE continent IS NOT NULL 
ORDER BY 1,2
"""
globalDeaths =  psql.sqldf(query)
globalDeaths.to_excel('/Users/benkelly/Downloads/Projects/COVID Overview/Data/1 Global Deaths.xlsx')

In [9]:
# Vaccinations by population

#Use CTE
query="""
WITH PopvsVax (Continent, Location, Date, Population,New_Vaccinations, RollingVaxCount) 
    AS (SELECT deaths.continent,deaths.location,deaths.date,deaths.population, 
    vax.new_vaccinations, SUM(vax.new_vaccinations) 
        OVER (PARTITION BY deaths.location 
              ORDER BY deaths.location, deaths.date) AS rollingVaxCount 
    FROM deaths 
    JOIN vax ON deaths.location=vax.location and deaths.date=vax.date 
    WHERE deaths.continent IS NOT NULL) 
SELECT *,(RollingVaxCount/Population)*100 AS RollingVaxPercent 
FROM PopvsVax
"""
popVax=psql.sqldf(query)
popVax

,Continent,Location,Date,Population,New_Vaccinations,RollingVaxCount,RollingVaxPercent
0,Asia,Afghanistan,2020-02-24,38928341.0,NaN,NaN,NaN
1,Asia,Afghanistan,2020-02-25,38928341.0,NaN,NaN,NaN
2,Asia,Afghanistan,2020-02-26,38928341.0,NaN,NaN,NaN
3,Asia,Afghanistan,2020-02-27,38928341.0,NaN,NaN,NaN
4,Asia,Afghanistan,2020-02-28,38928341.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
100599,Africa,Zimbabwe,2021-07-23,14862927.0,54959.0,2043414.0,13.748396
100600,Africa,Zimbabwe,2021-07-24,14862927.0,44604.0,2088018.0,14.048498
100601,Africa,Zimbabwe,2021-07-25,14862927.0,10738.0,2098756.0,14.120745
100602,Africa,Zimbabwe,2021-07-26,14862927.0,51307.0,2150063.0,14.465946


In [11]:
# Example of a temp table if pandasql if CREATE TABLE functioned normally
# I beleive the issues arise with setting the data types
query="""
CREATE TABLE PercentVax (Continent nvarchar(255), Location nvarchar(255), Date datetime, 
                        Population numeric, New_Vaccinations numeric, RollingVaxCount numeric) 
INSERT INTO PercentVax 
    SELECT deaths.continent,deaths.location,deaths.date,deaths.population, vax.new_vaccinations, 
        SUM(vax.new_vaccinations) 
            OVER (PARTITION BY deaths.location 
                  ORDER BY deaths.location, deaths.date) AS rollingVaxCount 
        FROM deaths 
        JOIN vax ON deaths.location=vax.location and deaths.date=vax.date 
        WHERE deaths.continent IS NOT NULL 

SELECT *,(RollingVaxCount/Population)*100 AS RollingVaxPercent FROM #PercentVax
"""
popVax=psql.sqldf(query)



PandaSQLException: (sqlite3.OperationalError) near "INSERT": syntax error
[SQL: 
CREATE TABLE PercentVax (Continent nvarchar(255), Location nvarchar(255), Date datetime, 
                        Population numeric, New_Vaccinations numeric, RollingVaxCount numeric) 
INSERT INTO PercentVax 
    SELECT deaths.continent,deaths.location,deaths.date,deaths.population, vax.new_vaccinations, 
        SUM(vax.new_vaccinations) 
            OVER (PARTITION BY deaths.location 
                  ORDER BY deaths.location, deaths.date) AS rollingVaxCount 
        FROM deaths 
        JOIN vax ON deaths.location=vax.location and deaths.date=vax.date 
        WHERE deaths.continent IS NOT NULL 

SELECT *,(RollingVaxCount/Population)*100 AS RollingVaxPercent FROM #PercentVax
]
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [13]:
#Creating View for later visualizations
#Saving it to a variable would be the pandasql version of these views. 
query="""
CREATE VIEW PercentPopVaxxed AS 
SELECT deaths.continent,deaths.location,deaths.date,deaths.population, vax.new_vaccinations, 
        SUM(vax.new_vaccinations) 
            OVER (PARTITION BY deaths.location 
                  ORDER BY deaths.location, deaths.date) AS rollingVaxCount 
FROM deaths 
JOIN vax ON deaths.location=vax.location and deaths.date=vax.date 
WHERE deaths.continent IS NOT NULL
"""
view=psql.sqldf(query)
view

In [14]:
#Total Deaths
query="""
SELECT location, SUM(new_deaths) as TotalDeathCount FROM deaths 
WHERE continent IS NULL 
GROUP BY location 
ORDER BY TotalDeathCount DESC
"""
tdeaths=psql.sqldf(query)
tdeaths.to_excel('/Users/benkelly/Downloads/Projects/COVID Overview/Data/2 Death Totals.xlsx')